In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
from subprocess import check_call
from glob import glob

import socket

import yaml
import textwrap

import cesm_tools
import util

this_notebook = 'create-new-case.ipynb'
assert 'cheyenne' in socket.gethostname(), (
    'this notebook must be run on cheyenne login node'
)

In [3]:
remote = 'git@github.com:ESCOMP/CESM.git'

coderoot = f"/glade/work/{os.environ['USER']}/codes"
cesmtag = 'release-cesm2.1.3'

cesmroot = cesm_tools.code_checkout(remote, coderoot, cesmtag)

In [4]:
USER = os.environ["USER"]
scriptroot = os.getcwd()

machine = "cheyenne"
if machine in ["cheyenne"]:
    inputdata = "/glade/p/cesmdata/cseg/inputdata"
    scratch = f"/glade/scratch/{USER}"
    project_root = f'/glade/work/{USER}/cmip6-cplhist'
    project_data = f'{project_root}/data'
    caseroot_root = f'{project_root}/cases'
else:
    raise ValueError(f"unknown machine: {machine}")

In [5]:
clobber = False

PROJECT = "P93300670"

cplhist_case = "b.e21.BHIST.f09_g17.CMIP6-historical.011"

cycle_ocn_physics = False
transient_forcing = False

member_id = 1

description = """
CPLHIST 1850 control with salinity restoring. 
First full test with S restoring operational.
"""

if cycle_ocn_physics:
    assert not transient_forcing, (
        "PHYS_CYCLE not setup to work with transient forcing"
    )
    phys_cycle = '.phys_cycle'
    compset = '1850_DATM%CPLHIST_SLND_CICE_POP2%ECO%PHYS-CYCLE_DROF%CPLHIST_SGLC_WW3'
else:
    phys_cycle = '.trnst' if transient_forcing else '.ctrl'
    compset = "1850_DATM%CPLHIST_SLND_CICE_POP2%ECO_DROF%CPLHIST_SGLC_WW3"

case = f'g.bforced{phys_cycle}{cplhist_case[1:]}.{member_id:03d}'
resolution = "f09_g17"

# initialization/forcing
run_startdate = '1850-01-01'
datm_cplhist_yr_start = 1850
datm_cplhist_yr_end = 1870

refcase = 'b.e21.B1850.f09_g17.CMIP6-piControl.001'
refdate = "0871-01-01"

# run setting
resubmit = 2
stop_n = 10
stop_option = "nyear"
rest_n = 10
rest_option = "nyear"
wallclock = "12:00:00"
queue = "regular"

# paths
caseinfo_file = f"case-info/{case}.yml"
caseroot = f"{caseroot_root}/{case}"
rundir = f"{scratch}/{case}"
archive_root = f"{scratch}/archive/{case}"

In [6]:
# ensure root directories
os.makedirs(os.path.dirname(caseroot), exist_ok=True)
os.makedirs(os.path.dirname(caseinfo_file), exist_ok=True)

if clobber:
    check_call(['rm', '-f', caseinfo_file])

create_build_case = not os.path.exists(caseinfo_file)

if create_build_case:
    with open(caseinfo_file, "w") as fid:
        yaml.dump(
            dict(
                case=case,
                description=description,
                caseroot=caseroot,
                rundir=rundir,
                archive_root=archive_root,
                compset=compset,
                resolution=resolution,
            ),
            fid,
        )
    if os.path.exists(caseroot):
        print(f"deleting {caseroot}")
        check_call(['rm', '-fr', caseroot])
    if os.path.exists(rundir):
        print(f"deleting {rundir}")
        check_call(['rm', '-fr', rundir])

In [7]:
if create_build_case:
    check_call(
        [
            "./create_newcase",
            "--case",
            caseroot,
            "--project",
            PROJECT,
            "--machine",
            machine,
            "--res",
            resolution,
            "--compset",
            compset,
            "--run-unsupport",
        ],
        cwd=f"{cesmroot}/cime/scripts",
    )

In [8]:
def xmlchange(arg):
    """call xmlchange"""
    check_call(["./xmlchange", arg], cwd=caseroot)

In [9]:
if create_build_case:

    # forcing/forcing collections
    xmlchange("OCN_COUPLING=full")

    if not transient_forcing:
        xmlchange("OCN_TRANSIENT=1850-2000")
        xmlchange("DATM_CO2_TSERIES=omip") # this might not be necessary as 
                                           # CO2 is in the cplhist ha2x3h files
                                           # TODO: test setting atm_co2_opt = 'drv_diag'
                                           #       and eliminate this setting

    # cplhist settings
    xmlchange(f"DATM_CPLHIST_CASE={cplhist_case}")
    xmlchange(f"DATM_CPLHIST_DIR={util.cplhist_stage_root}/cpl_hist/{cplhist_case}/monthly")
    xmlchange(f"DATM_CPLHIST_YR_START={datm_cplhist_yr_start}")
    xmlchange(f"DATM_CPLHIST_YR_END={datm_cplhist_yr_end}")
    xmlchange(f"DATM_CPLHIST_YR_ALIGN={datm_cplhist_yr_start}")

    xmlchange(f"DROF_CPLHIST_CASE={cplhist_case}")
    xmlchange(f"DROF_CPLHIST_DIR={util.cplhist_stage_root}/cpl_hist/{cplhist_case}/monthly")
    xmlchange(f"DROF_CPLHIST_YR_START={datm_cplhist_yr_start}")
    xmlchange(f"DROF_CPLHIST_YR_END={datm_cplhist_yr_end}")
    xmlchange(f"DROF_CPLHIST_YR_ALIGN={datm_cplhist_yr_start}")

    xmlchange(f"RUN_STARTDATE={run_startdate}")

else:
    xmlchange("CONTINUE_RUN=TRUE")

xmlchange(f"RESUBMIT={resubmit}")
xmlchange(f"STOP_OPTION={stop_option}")
xmlchange(f"STOP_N={stop_n}")
xmlchange(f"REST_OPTION={rest_option}")
xmlchange(f"REST_N={rest_n}")
xmlchange(f"JOB_WALLCLOCK_TIME={wallclock}")
xmlchange(f"JOB_QUEUE={queue}");

In [10]:
if create_build_case:
    check_call(['./case.setup'], cwd=caseroot)

In [11]:
src_pop_files = glob(f"{scriptroot}/SourceMods/src.pop/*")
for src in src_pop_files:
    src_basename = os.path.basename(src)
    dst = f"{caseroot}/SourceMods/src.pop/{src_basename}"
    shutil.copyfile(src, dst)

In [12]:
salinity_restoring_data_file = None
if not cycle_ocn_physics:
    salinity_restoring_data_file = glob(
        f"{util.restoring_data_stage_root}/{cplhist_case}/{cplhist_case}.SSS.day.*.nc"
    )

    assert len(salinity_restoring_data_file) == 1, (
        f"glob for: {util.restoring_data_stage_root}/{cplhist_case}/{cplhist_case}.SSS.day.*.nc",
        f"returned unexepected number of files: {len(salinity_restoring_data_file)}"

    )
    salinity_restoring_data_file = salinity_restoring_data_file[0]

salinity_restoring_data_file

In [13]:
def list_to_nml_list(lst):
    return ",".join([f"'{v}'" for v in lst])

user_nl = dict()

if cycle_ocn_physics:
    user_nl['pop'] = textwrap.dedent(
        f"""\
    &ecosys_forcing_data_nml
     atm_co2_opt = 'const'
     atm_co2_const = 284.7
     atm_alt_co2_const = 284.7
    """
    )
elif not transient_forcing:
    # add salinity restoring details to namelist
    user_nl['pop'] = textwrap.dedent(
        f"""\
    &ecosys_forcing_data_nml
     atm_co2_opt = 'const'
     atm_co2_const = 284.7
     atm_alt_co2_const = 284.7
    &forcing_s_interior_nml
     s_interior_data_type = 'shr_stream'
     s_interior_interp_freq = 'every-timestep'
     s_interior_variable_restore = .false.
     s_interior_surface_restore = .true.
     s_interior_interp_type = 'linear'
     s_interior_restore_tau = 1.0 ! days
     s_interior_data_renorm = 0.001
     s_interior_shr_stream_file = '{salinity_restoring_data_file}'
     s_interior_shr_stream_year_align = {datm_cplhist_yr_start}
     s_interior_shr_stream_year_first = {datm_cplhist_yr_start}
     s_interior_shr_stream_year_last = {datm_cplhist_yr_end}
    /
    """
    )
else:
    # add salinity restoring details to namelist
    user_nl['pop'] = textwrap.dedent(
        f"""\
    &ecosys_forcing_data_nml
     atm_alt_co2_opt = 'const'
     atm_alt_co2_const = 284.7
    &forcing_s_interior_nml
     s_interior_data_type = 'shr_stream'
     s_interior_interp_freq = 'every-timestep'
     s_interior_variable_restore = .false.
     s_interior_surface_restore = .true.
     s_interior_interp_type = 'linear'
     s_interior_restore_tau = 1.0 ! days
     s_interior_data_renorm = 0.001
     s_interior_shr_stream_file = '{salinity_restoring_data_file}'
     s_interior_shr_stream_year_align = {datm_cplhist_yr_start}
     s_interior_shr_stream_year_first = {datm_cplhist_yr_start}
     s_interior_shr_stream_year_last = {datm_cplhist_yr_end}
    /
    """
    )

if create_build_case:
    for key, nl in user_nl.items():
        with open(f"{caseroot}/user_nl_{key}", 'w') as fid:
            fid.write(user_nl[key])

In [14]:
if create_build_case:
    check_call(['qcmd', '-A', PROJECT, '--', './case.build'], cwd=caseroot)

In [15]:
# copy restarts
check_call(
    f"cp {util.restart_stage_root}/{refcase}/{refdate}-00000/* {rundir}/run/.",
    shell=True,
)

In [16]:
check_call(['./preview_namelists'], cwd=caseroot)

In [17]:
check_call(['./case.submit'], cwd=caseroot)